In [ ]:
import talib

In [ ]:
api_key = open('/tmp/.secret/alphavantage_api_key.txt').read()

In [ ]:
symbol = "RTX"

In [ ]:
url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&apikey={api_key}"

In [ ]:
import requests
res = requests.get(url)
print(res.status_code)

In [ ]:
raw_data:dict = res.json()

In [ ]:
#Convert alphavantage time series to pandas dataframe
import pandas as pd
res_df = pd.DataFrame(raw_data["Time Series (Daily)"],dtype=float).T

In [ ]:
#convert panda dataframe index to datetime
res_df.index = pd.to_datetime(res_df.index)
#calculate talib SMA from pandas dataframe
res_df["SMA10"] = talib.SMA(res_df["4. close"].astype(float), timeperiod=10).astype(float)
res_df["SMA200"] = talib.SMA(res_df["4. close"].astype(float), timeperiod=200).astype(float)
#calculate talib RSI from pandas dataframe
res_df["RSI"] = talib.RSI(res_df["4. close"].astype(float), timeperiod=14).astype(float)
res_df.dtypes

In [ ]:
#PLotly graph with candlestick and sma
#Make plotly subplot of primary and secondary y axis
from plotly.subplots import make_subplots
import plotly.graph_objects as go
fig = make_subplots(rows=2, cols=1,
                        specs=[[{"secondary_y": True}],
                             [{"secondary_y": True}]],
                        shared_xaxes=True,
                        row_heights=[10, 8])

# #Plot the close price
# fig = px.line(res_df, x=res_df.index, y=["4. close",  "SMA10", "SMA200"])

#Plot the candlestick
fig.add_trace(
    go.Candlestick(
        x=res_df.index,
        open=res_df["1. open"],
        high=res_df["2. high"],
        low=res_df["3. low"],
        close=res_df["4. close"],
        name="Candlestick"
    ),row=1,col=1
)

#subplot RSI in a separate graph
fig.add_scatter(x=res_df.index, y=res_df["RSI"], row=2, col=1
                 ,name="RSI")



# fig.update_yaxes(range=[ 80,110]) 
# print(fig.layout)
fig.update_layout(margin=dict(l=20, r=20, t=20, b=20),
        paper_bgcolor="LightSteelBlue",
        showlegend=False,
        xaxis= {'rangeslider': {'visible': False},
              'showticklabels': False},
        xaxis2= {'rangeslider': {'visible': True}},
        yaxis = dict(title="candles", range=[80,110]),
        yaxis3 = dict(title="RSI", range=[0,80]),
)

# fig.update_layout(width=1600, height=400)
# fig.update_xaxes(rangeslider_visible=True)
#fig.update_yaxes(range=[0,100])
print(fig.layout)
fig.show()



In [ ]:
print(fig.layout)